## 路由（Router）
LEL ( LangChain Expression Language ) 实现了路由机制，支持在应用开发中根据业务需要将请求转发给指定的链。

核心类：RouterRunnable

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap, Runnable, RunnableConfig, RunnablePassthrough
from langchain_core.runnables.utils import Input
from langchain.load.serializable import Serializable
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from operator import itemgetter
from typing import Optional, Dict

一个有用的调试类 StdOutputRunnable
类 StdOutputRunnable 的功能与 RunnablePassthrough 几乎一致，唯一的区别是打印出输入的数据。这非常适合于 LEL 的调试。将其加到管道中，可以查看所在环节的输入值，帮助调试。

In [3]:
class StdOutputRunnable(Serializable, Runnable[Input, Input]):
    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Dict, config: Optional[RunnableConfig] = None) -> Input:
        print(input)
        return self._call_with_config(lambda x: x, input, config)

## RouterRunnable 示例
模型辅助类，在分类任务执行时，指定允许的分类值。

create_tagging_chain_pydantic 创建基于 Pydantic schema定义的分类链，

In [5]:
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field

from dotenv import load_dotenv
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI
from langchain_community.llms.tongyi import Tongyi
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')
print(DASHSCOPE_API_KEY)

# qwen
qwen_llm = Tongyi(
    model='qwen2-1.5b-instruct'
)

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn
sk-01c5003340c3453b934052d737d45e01
sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [6]:
class ChainToUse(BaseModel):
    """Used to determine which chain to serve the user.用于确定为用户提供哪条链"""

    name: str = Field(description="Should be one of `color` or `fruit`")


tagger = create_tagging_chain_pydantic(ChainToUse, cf_llm)

ValidationError: 2 validation errors for PydanticOutputFunctionsParser
pydantic_schema
  subclass of BaseModel expected (type=type_error.subclass; expected_class=BaseModel)
pydantic_schema
  value is not a valid dict (type=type_error.dict)

In [ ]:
color_chain = ChatPromptTemplate.from_template(
    "You are a color expert. Answer the question about color: {question}") | cf_llm
fruit_chain = ChatPromptTemplate.from_template(
    "You are a fruit expert. Answer the question about fruit: {question}") | cf_llm

In [ ]:
from langchain.schema.runnable import RouterRunnable

router = RouterRunnable({"color": color_chain, "fruit": fruit_chain})

In [ ]:
chain = {
            "key": RunnablePassthrough()
                   | tagger
                   | StdOutputRunnable()
                   | (lambda x: x['text'].name), "input": {"question": RunnablePassthrough()}
        } | router

In [ ]:
chain.invoke("What is the HEX code of color YELLOW?")

In [ ]:
chain.invoke("What country grow most apples in 2000?")